In [1]:
import pandas as pd
import numpy as np
from random import sample
import matplotlib.pyplot as plt
from IPython.core.pylabtools import figsize
import glob
import cv2

#from google.colab import drive

import matplotlib.pyplot as plt
import keras as ks
from keras import layers
from keras.models import Sequential
import tensorflow as tf
from keras.backend import categorical_crossentropy

import os

from keras.applications.inception_v3 import preprocess_input
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from keras import applications
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import SGD

In [2]:
#drive.mount('/content/drive')

In [3]:
image_size = (250, 250)
batch_size = 32


train_directory = '5 classes/Training'
val_directory = '5 classes/Valid'
test_directory = '5 classes/Test'

train_ds = tf.keras.preprocessing.image_dataset_from_directory(train_directory, label_mode='categorical', image_size=image_size, batch_size=batch_size)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(val_directory, label_mode='categorical', image_size=image_size, batch_size=batch_size)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(test_directory, label_mode='categorical', image_size=image_size, batch_size=batch_size)

train_ds = train_ds.prefetch(buffer_size=batch_size)
val_ds = val_ds.prefetch(buffer_size=batch_size)
test_ds = test_ds.prefetch(buffer_size=batch_size)

Found 25000 files belonging to 5 classes.
Found 27427 files belonging to 5 classes.
Found 34281 files belonging to 5 classes.


In [4]:
input_shape = (250, 250)
nclass = 5

# create the base pre-trained model 
base_model = InceptionV3(weights='imagenet', 
                                include_top=False, 
                                input_shape= (input_shape+(3,)))

# to freeze the base model 
# https://keras.io/guides/transfer_learning/
#base_model.trainable = False

# to add layers to the model 
add_model = Sequential()
add_model.add(base_model)
add_model.add(layers.Rescaling(1.0/255))
add_model.add(layers.Conv2D(32, 3, strides=2, padding='same'))
add_model.add(layers.BatchNormalization())
add_model.add(layers.Activation('relu'))
add_model.add(layers.Conv2D(64, 3, strides=2, padding='same'))
add_model.add(layers.BatchNormalization())
add_model.add(layers.Activation('relu'))
add_model.add(GlobalAveragePooling2D()) # to add a global spatial average pooling layer 
add_model.add(Dropout(0.5))
add_model.add(Dense(nclass, activation='softmax')) #there are 5 classes 

# compile the model (should be done *after* setting layers to non-trainable)
model = add_model
model.compile(optimizer='adam', loss='CategoricalCrossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 6, 6, 2048)        21802784  
                                                                 
 rescaling (Rescaling)       (None, 6, 6, 2048)        0         
                                                                 
 conv2d_94 (Conv2D)          (None, 3, 3, 32)          589856    
                                                                 
 batch_normalization_94 (Bat  (None, 3, 3, 32)         128       
 chNormalization)                                                
                                                                 
 activation_94 (Activation)  (None, 3, 3, 32)          0         
                                                                 
 conv2d_95 (Conv2D)          (None, 2, 2, 64)          18496     
                                                        

In [5]:
file_path="5 classes/inceptionv3_5C5K_{epoch}.h5"

epochs = 1000

callbacks = [ks.callbacks.ModelCheckpoint(file_path, save_best_only=True), 
             ks.callbacks.EarlyStopping(patience=20, restore_best_weights=True)]

history = model.fit(train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds)

Epoch 1/1000
782/782 [==============================] - 3609s 5s/step - loss: 0.9012 - accuracy: 0.6752 - val_loss: 3.5578 - val_accuracy: 0.2230
Epoch 2/1000
782/782 [==============================] - 3627s 5s/step - loss: 0.5919 - accuracy: 0.8050 - val_loss: 1.6133 - val_accuracy: 0.5063
Epoch 3/1000
782/782 [==============================] - 3714s 5s/step - loss: 0.4974 - accuracy: 0.8383 - val_loss: 2.9787 - val_accuracy: 0.3544
Epoch 4/1000
782/782 [==============================] - 3511s 4s/step - loss: 0.4478 - accuracy: 0.8557 - val_loss: 2.2987 - val_accuracy: 0.5100
Epoch 5/1000
782/782 [==============================] - 3496s 4s/step - loss: 0.3883 - accuracy: 0.8754 - val_loss: 0.9999 - val_accuracy: 0.7483
Epoch 6/1000
782/782 [==============================] - 3484s 4s/step - loss: 0.3451 - accuracy: 0.8886 - val_loss: 1.3928 - val_accuracy: 0.6309
Epoch 7/1000
782/782 [==============================] - 3484s 4s/step - loss: 0.3124 - accuracy: 0.9006 - val_loss: 1.1785 -

KeyboardInterrupt: 

In [ ]:
true = []
pred = []
batches = int(np.ceil(34281/32))
i = 1
for image, label in test_ds:
  pred = np.concatenate([pred, 
                         np.argmax(model.predict(image), axis=-1)])
  true = np.concatenate([true,
                         np.argmax(label.numpy(), axis=-1)])
  print('{}/{}'.format(i, batches))
  i += 1

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(true, pred)

In [ ]:
model_results = pd.DataFrame({'true':true,
                              'pred':pred})

model_results.to_csv('5 classes/inceptionv3_v3_ST_test_preds.csv')